In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
import matplotlib.pyplot as plt
# tf.random.set_seed(123)
# np.random.seed(123)
from sklearn import metrics
import json 
import os

def timeseries_evaluation_metrics_func(y_true, y_pred):
    
    def mean_absolute_percentage_error(y_true, y_pred): 
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Evaluation metric results:-')
    mse = metrics.mean_squared_error(y_true, y_pred)
    mae = metrics.mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(metrics.mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)
    print(f'MSE is : {mse}')
    print(f'MAE is : {mae}')
    print(f'RMSE is : {rmse}')
    print(f'MAPE is : {mape}')
    print(f'R2 is : {r2}',end='\n\n')
    return {'mse' : mse, 'mae' : mae, 'rmse' : rmse, 'mape' : mape, 'r2' : r2}

# df = pd.read_csv(r'..\Data\Panama Electricity\continuous_dataset_preprocessing.csv', parse_dates= True)
df = pd.read_csv(
    "../Data/Panama Electricity/continuous_dataset_preprocessing.csv",
    parse_dates=True,
)
df["datetime"] = df["datetime"].apply(pd.to_datetime)

column_names = df.columns.tolist()
train_column_names = df.columns.tolist()
column_names.remove('datetime')
train_column_names.remove('datetime')
train_column_names.remove('nat_demand')

In [3]:
tf.keras.backend.clear_session()
# day_to_test = '1/1/2019'
for month in [*range(1, 12)]:
    day = f'1/{month}/2019'
    valid = df[df['datetime'] <= '1/1/2019 23:00']
    valid = valid[valid['datetime'] > '1/1/2018 23:00']
    train = df[df['datetime'] <= '1/1/2018 23:00']
    test = df[df['datetime'] > day + ' 23:00']
    datatest = df[df['datetime'] <= day + ' 23:00']
    version = 1
    hist_window = 42 # 12, 18, 24, 30, 36, 42, 48, 54, 60
    horizon = 24 # 12, 18, 24, 30, 36
    folder_save = f'../Results/GRU/history_{hist_window}_future_{horizon}_version_{version}'

    model_path = f'{folder_save}/model.h5' 

    x_scaler = preprocessing.MinMaxScaler()
    y_scaler = preprocessing.MinMaxScaler()
    validX = x_scaler.fit_transform(valid[column_names])
    validY = y_scaler.fit_transform(valid[['nat_demand']])

    Trained_model = tf.keras.models.load_model(model_path)
    folder_save = f'../Results/GRU/{month}'
    isExist = os.path.exists(folder_save)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(folder_save)
       print("The new directory is created!")

    data_test = x_scaler.fit_transform(datatest[column_names].tail(hist_window))
    test_rescaled = data_test.reshape(1, data_test.shape[0], data_test.shape[1])

    Predicted_results = Trained_model.predict(test_rescaled)
    Predicted_results_Inv_trans = y_scaler.inverse_transform(Predicted_results)
    metrics_test = timeseries_evaluation_metrics_func(test['nat_demand'][:horizon],Predicted_results_Inv_trans[0])
    with open(f"{folder_save}/metrics_test.json", "w") as outfile:
        json.dump(metrics_test, outfile)

The new directory is created!
1/1 [==============================] - 1s 889ms/step
Evaluation metric results:-
MSE is : 3650.2973486872593
MAE is : 52.13602635091146
RMSE is : 60.4176906931013
MAPE is : 4.635879672183831
R2 is : 0.9111442117401747

The new directory is created!
1/1 [==============================] - 1s 867ms/step
Evaluation metric results:-
MSE is : 5995.583593784115
MAE is : 59.801669230143226
RMSE is : 77.43115389676248
MAPE is : 5.062179127914524
R2 is : 0.8344138422633229

The new directory is created!
1/1 [==============================] - 1s 927ms/step
Evaluation metric results:-
MSE is : 13711.847599011475
MAE is : 92.39930034993489
RMSE is : 117.09759860480263
MAPE is : 7.836099703865878
R2 is : 0.6261160396461398

The new directory is created!
1/1 [==============================] - 1s 1s/step
Evaluation metric results:-
MSE is : 22193.224427853667
MAE is : 110.65345134277341
RMSE is : 148.97390519098863
MAPE is : 9.840035350590192
R2 is : -0.3703597295573462



In [2]:
tf.keras.backend.clear_session()
# day_to_test = '1/1/2019'
for month in [*range(1, 12)]:
    day = f'1/{month}/2019'
    valid = df[df['datetime'] <= '1/1/2019 23:00']
    valid = valid[valid['datetime'] > '1/1/2018 23:00']
    train = df[df['datetime'] <= '1/1/2018 23:00']
    test = df[df['datetime'] > day + ' 23:00']
    datatest = df[df['datetime'] <= day + ' 23:00']
    version = 1
    hist_window = 30 # 12, 18, 24, 30, 36, 42, 48, 54, 60
    horizon = 24 # 12, 18, 24, 30, 36
    folder_save = f'../Results/Attention_LSTM/history_{hist_window}_future_{horizon}_version_{version}'

    model_path = f'{folder_save}/model.h5' 

    x_scaler = preprocessing.MinMaxScaler()
    y_scaler = preprocessing.MinMaxScaler()
    validX = x_scaler.fit_transform(valid[column_names])
    validY = y_scaler.fit_transform(valid[['nat_demand']])

    Trained_model = tf.keras.models.load_model(model_path)
    folder_save = f'../Results/Attention_LSTM/{month}'
    isExist = os.path.exists(folder_save)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(folder_save)
       print("The new directory is created!")

    data_test = x_scaler.fit_transform(datatest[column_names].tail(hist_window))
    test_rescaled = data_test.reshape(1, data_test.shape[0], data_test.shape[1])

    Predicted_results = Trained_model.predict(test_rescaled)
    Predicted_results_Inv_trans = y_scaler.inverse_transform(Predicted_results)
    metrics_test = timeseries_evaluation_metrics_func(test['nat_demand'][:horizon],Predicted_results_Inv_trans[0])
    with open(f"{folder_save}/metrics_test.json", "w") as outfile:
        json.dump(metrics_test, outfile)

The new directory is created!
1/1 [==============================] - 1s 1s/step
Evaluation metric results:-
MSE is : 6902.931241021285
MAE is : 72.89999655761716
RMSE is : 83.08388075325614
MAPE is : 6.019486225621605
R2 is : 0.8319683745914829

The new directory is created!
1/1 [==============================] - 1s 1s/step
Evaluation metric results:-
MSE is : 7078.911461184413
MAE is : 65.83647433268227
RMSE is : 84.1362672168454
MAPE is : 5.695012102753066
R2 is : 0.8044944697241995

The new directory is created!
1/1 [==============================] - 1s 1s/step
Evaluation metric results:-
MSE is : 4997.490867156178
MAE is : 58.1563546508789
RMSE is : 70.69293364372551
MAPE is : 4.9053658334194745
R2 is : 0.8637323188029524

The new directory is created!
1/1 [==============================] - 1s 1s/step
Evaluation metric results:-
MSE is : 15635.420022473407
MAE is : 103.41971295979816
RMSE is : 125.04167314329014
MAPE is : 8.844930101242587
R2 is : 0.034563453221280294

The new dire

In [5]:
tf.keras.backend.clear_session()
# day_to_test = '1/1/2019'
for month in [*range(1, 12)]:
    day = f'1/{month}/2019'
    valid = df[df['datetime'] <= '1/1/2019 23:00']
    valid = valid[valid['datetime'] > '1/1/2018 23:00']
    train = df[df['datetime'] <= '1/1/2018 23:00']
    test = df[df['datetime'] > day + ' 23:00']
    datatest = df[df['datetime'] <= day + ' 23:00']
    version = 1
    hist_window = 42 # 12, 18, 24, 30, 36, 42, 48, 54, 60
    horizon = 24 # 12, 18, 24, 30, 36
    folder_save = f'../Results/GRU_clip/history_42_future_24_version_700'

    model_path = f'{folder_save}/model.h5' 

    x_scaler = preprocessing.MinMaxScaler()
    y_scaler = preprocessing.MinMaxScaler()
    validX = x_scaler.fit_transform(valid[column_names])
    validY = y_scaler.fit_transform(valid[['nat_demand']])

    Trained_model = tf.keras.models.load_model(model_path)
    folder_save = f'../Results/GRU_clip/{month}'
    isExist = os.path.exists(folder_save)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(folder_save)
       print("The new directory is created!")

    data_test = x_scaler.fit_transform(datatest[column_names].tail(hist_window))
    test_rescaled = data_test.reshape(1, data_test.shape[0], data_test.shape[1])

    Predicted_results = Trained_model.predict(test_rescaled)
    Predicted_results_Inv_trans = y_scaler.inverse_transform(Predicted_results)
    metrics_test = timeseries_evaluation_metrics_func(test['nat_demand'][:horizon],Predicted_results_Inv_trans[0])
    with open(f"{folder_save}/metrics_test.json", "w") as outfile:
        json.dump(metrics_test, outfile)

1/1 [==============================] - 1s 856ms/step
Evaluation metric results:-
MSE is : 10222.740045912695
MAE is : 83.18279296468097
RMSE is : 101.10756670948369
MAPE is : 6.528369537348228
R2 is : 0.7511573611170275

The new directory is created!
1/1 [==============================] - 1s 874ms/step
Evaluation metric results:-
MSE is : 10914.729523577294
MAE is : 81.64771701253254
RMSE is : 104.47358289815323
MAPE is : 6.63249862925091
R2 is : 0.6985567632785619

The new directory is created!
1/1 [==============================] - 1s 860ms/step
Evaluation metric results:-
MSE is : 11493.531917588954
MAE is : 89.19762938232422
RMSE is : 107.2078911162278
MAPE is : 7.295610009225852
R2 is : 0.6866033405949279

The new directory is created!
1/1 [==============================] - 1s 950ms/step
Evaluation metric results:-
MSE is : 18987.4713536553
MAE is : 110.08033494059244
RMSE is : 137.79503384975564
MAPE is : 9.305443598480657
R2 is : -0.17241485993882333

The new directory is create